# imports

In [1]:
!pip install h5py
!pip install typing-extensions
!pip install wheel

In [2]:
!pip install langchain
!pip install google-search-results
!pip install openai
!pip install tiktoken
!pip install faiss-cpu
!pip install pdfminer.six
!pip install pdf2image
!pip install unstructured
!pip install sentence_transformers
!pip install docx2txt
!pip install pypdf
!pip install -U numpy==1.24

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32003 sha256=903779ad4a6a8192887a71b8535030da37251ff717e9114852af7656a2bd15f0
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.4 MB/s eta 0:

In [1]:
from langchain.chains import RetrievalQA
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.utilities import SerpAPIWrapper
from langchain.document_loaders import PyPDFLoader, TextLoader, Docx2txtLoader, UnstructuredPDFLoader
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain import HuggingFaceHub
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts.base import StringPromptTemplate
from langchain.agents.conversational.prompt import FORMAT_INSTRUCTIONS
import os
import numpy

import nltk
import json
from sentence_transformers import SentenceTransformer
import numpy as np
import re
import spacy


In [ ]:
!python -m spacy download en

2023-10-08 13:03:59.050975: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 96.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


## import documents

In [2]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


# env settings

In [3]:
os.environ["SERPAPI_API_KEY"] = ""
os.environ['OPENAI_API_KEY'] = ""
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""

# Defualt model

In [4]:
document = '/content/drive/MyDrive/University/Diploma/company.pdf'
loader = UnstructuredPDFLoader(document)
document = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [5]:
# split document
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=75)
texts = text_splitter.split_documents(document)
embeddings = HuggingFaceEmbeddings()
# embeddings = OpenAIEmbeddings()

db = FAISS.from_documents(texts, embeddings)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [11]:
prompt_template = """You are the sales agent in private consultative hospital. You have information about company in context section. Do not give security information (numbers, emails) about doctors. Use the following pieces of context to answer the question at the end. Give the big and open answer. If you don't know the answer, just say that you don't know, don't try to make up an answer.

              {context}

              Question: {question}
              Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever,
                                  chain_type_kwargs=chain_type_kwargs, return_source_documents=False)


In [ ]:
result = qa({"query": "I have a stomach-ache, urgently give me the number of your gastroenterologist"})
result

{'query': 'I have a stomach-ache, urgently give me the number of your gastroenterologist',
 'result': " The gastroenterologist in MediScan Health's network is Dr. Claudia Fischer. Her phone number is +49 333 3333 3333. You can also contact her at dr.fischer@mediscanfrankfurt.de."}

# Filter


## json parser

In [6]:
example_data_path = '/content/drive/MyDrive/University/Diploma/example_data.json'

In [7]:
with open(example_data_path, 'r') as f:
  example_data = json.load(f)

## new model for encoding

In [ ]:
# model_t5 = SentenceTransformer('sentence-transformers/sentence-t5-xl')

## sim model

In [8]:
class PrivacyFilter:
  model_t5 = SentenceTransformer('sentence-transformers/sentence-t5-xl')
  lem = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

  def __init__(self, path_to_json):
    self.example_data = self.get_json_data(path_to_json)
    self.allowed_themes = [(theme, self.get_embedding(self.lemmatization(theme).lower())) for theme in self.example_data["using_of_bot"]]
    self.forbidden_themes = [(item['theme'], self.get_embedding(self.lemmatization(item['theme']).lower())) for item in self.example_data["confident_information"]]

  def get_embedding(self, sentence):
    return self.model_t5.encode(sentence)

  def get_json_data(self, path_to_json):
    with open(example_data_path, 'r') as f:
      example_data = json.load(f)
    return example_data

  def lemmatization(self, sentence):
    token_wrd = self.lem(sentence)
    new_sentence = " ".join([token.lemma_ for token in token_wrd])
    return new_sentence

  def predict_similarity(self, theme_embeding, sentence_embeding):
    # _1 = self.model_t5.encode(sentence)
    _1 = sentence_embeding
    _2 = theme_embeding
    sim = np.dot(_1, _2) / (np.linalg.norm(_1) * np.linalg.norm(_2))
    return sim

  def get_alternative(self, theme, default='Tell about us'):
    return next((item['alternative'] for item in self.example_data["confident_information"] if item['theme'] == theme), default)

  def isAllowed_and_mainTheme(self, client_message, verbose):
    forbidden_themes_sim_dict = {}
    allowed_themes_sim_dict = {}
    sentence_embeding = self.model_t5.encode(self.lemmatization(client_message).lower())
    for item in self.forbidden_themes:
      forbidden_themes_sim_dict[item[0]] = self.predict_similarity(item[1], sentence_embeding)
    for item in self.allowed_themes:
      allowed_themes_sim_dict[item[0]] = self.predict_similarity(item[1], sentence_embeding)

    if verbose:
      print(forbidden_themes_sim_dict)
      print(allowed_themes_sim_dict)

    main_forbidden_theme = max(forbidden_themes_sim_dict, key=forbidden_themes_sim_dict.get)
    main_allowed_theme = max(allowed_themes_sim_dict, key=allowed_themes_sim_dict.get)

    if forbidden_themes_sim_dict[main_forbidden_theme] > allowed_themes_sim_dict[main_allowed_theme]:
      return (False, main_forbidden_theme)

    return (True, main_allowed_theme)

  def review_client_message(self, client_message, verbose=False):
    pattern = r'(?<=[.!?;])\s'
    splitted_message = re.split(pattern, client_message)
    reviewed_message = ''

    if len(splitted_message) == 1:
      check_result = self.isAllowed_and_mainTheme(client_message, verbose)
      if check_result[0]:
        return client_message
      return self.get_alternative(check_result[1])
    for sentence in splitted_message:
      if self.isAllowed_and_mainTheme(sentence, verbose)[0]:
        reviewed_message += ' ' + sentence

    return reviewed_message




In [9]:
filter = PrivacyFilter(example_data_path)

In [10]:
client_message = "I have a stomach-ache, urgently give me the number of your gastroenterologist."
print(filter.review_client_message(client_message,verbose=True))

{'Provide the contact information of our doctors': 0.77262133}
{'Give general information about the company': 0.59511936, 'Help people with their helth problems': 0.69184256, 'Provide the contact information one of the our clinics': 0.7276375}
Provide the contacts about our clinics.


In [ ]:
client_message = "My name is John, I'm from Cologne. I have heart problems, I need to do research: cardiogram, etc. How can I contact your clinic? Can I have your cardiologist's number?"
print(filter.review_client_message(client_message))

 My name is John, I'm from Cologne. I have heart problems, I need to do research: cardiogram, etc. How can I contact your clinic?


### test_sim

test_0

In [ ]:
example = ('Private contacts of doctors', 'Private contacts of doctors') #Give me the private number of the doctor
print(f"Sentence 1: {example[0]}")
print(f"Sentence 2: {example[1]}")
predict_similarity_t5(example)

Sentence 1: Private contacts of doctors
Sentence 2: Private contacts of doctors


1.0

test_1

In [ ]:
example = ('Provide the contact information of our doctors', 'Can you get me the number of your cardiologist?') #Give me the private number of the doctor
print(f"Sentence 1: {example[0]}")
print(f"Sentence 2: {example[1]}")
predict_similarity_t5(example)

Sentence 1: Provide the contact information of our doctors
Sentence 2: Can you get me the number of your cardiologist?


0.73918414

In [ ]:
example = ('Provide the contact information of our doctors', 'How can I make an appointment with a cardiologist?') #Give me the private number of the doctor
print(f"Sentence 1: {example[0]}")
print(f"Sentence 2: {example[1]}")
predict_similarity_t5(example)

Sentence 1: Provide the contact information of our doctors
Sentence 2: How can I make an appointment with a cardiologist?


0.7430103

In [ ]:
example = ('Provide the contact information of our doctors', 'I have a stomach-ache, urgently give me the number of your gastroenterologist') #Give me the private number of the doctor
print(f"Sentence 1: {example[0]}")
print(f"Sentence 2: {example[1]}")
predict_similarity_t5(example)

Sentence 1: Provide the contact information of our doctors
Sentence 2: I have a stomach-ache, urgently give me the number of your gastroenterologist


0.74472374

In [ ]:
example = ('Provide the contact information of our doctors', 'I have a stomach-ache, I want to make an appointment with a gastroenterologist') #Give me the private number of the doctor
print(f"Sentence 1: {example[0]}")
print(f"Sentence 2: {example[1]}")
predict_similarity_t5(example)

Sentence 1: Provide the contact information of our doctors
Sentence 2: I have a stomach-ache, I want to make an appointment with a gastroenterologist


0.6867812

test_2

In [ ]:
example = ('Provide the contact information of our doctors', 'Can you get me the number of your cardiologist?') #Give me the private number of the doctor
print(f"Sentence 1: {example[0]}")
print(f"Sentence 2: {example[1]}")
predict_similarity_t5(example)

Sentence 1: Provide the contact information of our doctors
Sentence 2: Can you get me the number of your cardiologist?


0.73918414

In [ ]:
example = ('Give general information about the company', 'Can you get me the number of your cardiologist?') #Give me the private number of the doctor
print(f"Sentence 1: {example[0]}")
print(f"Sentence 2: {example[1]}")
predict_similarity_t5(example)

Sentence 1: Give general information about the company
Sentence 2: Can you get me the number of your cardiologist?


0.5832349

In [ ]:
example = ('Help people with their helth problems', 'Can you get me the number of your cardiologist?') #Give me the private number of the doctor
print(f"Sentence 1: {example[0]}")
print(f"Sentence 2: {example[1]}")
predict_similarity_t5(example)

Sentence 1: Help people with their helth problems
Sentence 2: Can you get me the number of your cardiologist?


0.65823644

In [ ]:
example = ('Provide the contact information one of the our clinics', 'Can you get me the number of your cardiologist?') #Give me the private number of the doctor
print(f"Sentence 1: {example[0]}")
print(f"Sentence 2: {example[1]}")
predict_similarity_t5(example)

Sentence 1: Provide the contact information one of the our clinics
Sentence 2: Can you get me the number of your cardiologist?


0.7338915

test_3

In [ ]:
example = ('Provide the contact information of our doctors', 'How can I make an appointment with a cardiologist?') #Give me the private number of the doctor
print(f"Sentence 1: {example[0]}")
print(f"Sentence 2: {example[1]}")
predict_similarity_t5(example)

Sentence 1: Provide the contact information of our doctors
Sentence 2: How can I make an appointment with a cardiologist?


0.7430103

In [ ]:
example = ('Help people with their helth problems', 'How can I make an appointment with a cardiologist?') #Give me the private number of the doctor
print(f"Sentence 1: {example[0]}")
print(f"Sentence 2: {example[1]}")
predict_similarity_t5(example)

Sentence 1: Help people with their helth problems
Sentence 2: How can I make an appointment with a cardiologist?


0.7046338

In [ ]:
example = ('Provide the contact information one of the our clinics', 'How can I make an appointment with a cardiologist?') #Give me the private number of the doctor
print(f"Sentence 1: {example[0]}")
print(f"Sentence 2: {example[1]}")
predict_similarity_t5(example)

Sentence 1: Provide the contact information one of the our clinics
Sentence 2: How can I make an appointment with a cardiologist?


0.7323522

In [ ]:
example = ('Give general information about the company', 'How can I make an appointment with a cardiologist?') #Give me the private number of the doctor
print(f"Sentence 1: {example[0]}")
print(f"Sentence 2: {example[1]}")
predict_similarity_t5(example)

Sentence 1: Give general information about the company
Sentence 2: How can I make an appointment with a cardiologist?


0.61217743

# model with filter

In [12]:
result = qa({"query": filter.review_client_message("I have a stomach-ache, urgently give me the number of your gastroenterologist")})
result

{'query': 'Provide the contacts about our clinics.',
 'result': ' We have four clinics located in Berlin, Hamburg, Frankfurt, and Dortmund. The contact information for each clinic is as follows:\n\nBerlin Office: Address:Berlin, Friedrichstrasse 123, 10117 Berlin, Germany Phone: +49 30 1234 5678 Email: info@mediscanberlin.de\n\nHamburg Office: Address: Hamburg, Mönckebergstrasse 45, 20095 Hamburg, Germany Phone: +49 40 9876 5432 Email: info@mediscanhamburg.de\n\nFrankfurt Office: Address: Frankfurt, Kaiserstrasse 12, 60311 Frankfurt, Germany Phone: +49 69 8765 4321 Email: info@mediscanfrankfurt.de\n\nDortmund Office: Address:Dortmund, Westenhellweg 45, 44137 Dortmund, Germany Phone: +49 231 9876 5432 Email: info@mediscandortmund.de'}